In [1]:
import numpy as np
import pandas as pd
import re
import requests
from requests_oauthlib import OAuth1
import urllib
import datetime
import time
import json
from kafka import SimpleProducer, KafkaClient

import sys

from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils


In [2]:
BATCH_INTERVAL = 60  # How frequently to update (seconds)
BLOCKSIZE = 50  # How many tweets per update

In [4]:
import os
os.environ['SPARK_CLASSPATH']

KeyError: 'SPARK_CLASSPATH'

In [3]:
#start spark streaming context with sc
#note: you need to create sc if you're not running the ispark setup in ipython notebook
ssc = StreamingContext(sc, 1)

# create kafka streaming context
directKafkaStream = KafkaUtils.createDirectStream(ssc, ["tweets"], {"bootstrap.servers": "localhost:9092"})


________________________________________________________________________________________________

  Spark Streaming's Kafka libraries not found in class path. Try one of the following.

  1. Include the Kafka library and its dependencies with in the
     spark-submit command as

     $ bin/spark-submit --packages org.apache.spark:spark-streaming-kafka:1.5.1 ...

  2. Download the JAR of the artifact from Maven Central http://search.maven.org/,
     Group Id = org.apache.spark, Artifact Id = spark-streaming-kafka-assembly, Version = 1.5.1.
     Then, include the jar in the spark-submit command as

     $ bin/spark-submit --jars <spark-streaming-kafka-assembly.jar> ...

________________________________________________________________________________________________




Py4JJavaError: An error occurred while calling o20.loadClass.
: java.lang.ClassNotFoundException: org.apache.spark.streaming.kafka.KafkaUtilsPythonHelper
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:497)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:379)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:133)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:207)
	at java.

In [2]:
# To send messages synchronously
kafka = KafkaClient('localhost:9092')
producer = SimpleProducer(kafka)

In [3]:
APP_KEY = "bv6mnYBiFeEVKvPEZlg"
APP_SECRET = "nQZk9Ca8qqJxc1Za07WyW0VPZ6gtAUSF3oPD5sun0"
OAUTH_TOKEN = "606525030-ilOtJstbRvFCjUNMtOu8DP2HQKGWpQvmUsF6fblE"
OAUTH_TOKEN_SECRET = "xSVE47qVOFxxZm1oqKwL6zwLVMWpzxCUYGmLJ6CVHR0mZ"

config_token = OAuth1(APP_KEY,
                      client_secret=APP_SECRET,
                      resource_owner_key=OAUTH_TOKEN,
                      resource_owner_secret=OAUTH_TOKEN_SECRET)

config_url = 'https://stream.twitter.com/1.1/statuses/filter.json'

search_terms = np.loadtxt("search-terms.txt",delimiter="\n",dtype=object)
search_terms = ','.join(search_terms)
search_terms = urllib.urlencode({"track":search_terms}).split("=")[1]

In [4]:
data      = [('language', 'en'), ('track', search_terms)]
query_url = config_url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in data])
response  = requests.get(query_url, auth=config_token, stream=True)

In [30]:
hour = 13
minute = 40

In [31]:
if response.status_code == 200:
    ct = 0
    timesup = datetime.datetime(2015,11,11,hour,minute).strftime('%s')
    for line in response.iter_lines():  # Iterate over streaming tweets
        if int(timesup) > time.time():
            producer.send_messages('tweets', line.decode('utf8'))
            '''
            tweet = json.loads(line.decode('utf8'))
            contents = {"tweet":tweet['text'], 
                        "user":tweet['user']['screen_name'], 
                        "location":tweet['user']['location'],
                        "time zone":tweet['user']['time_zone'],
                        "coordinates":tweet['coordinates']
                        }
            
            producer.send_messages('tweets', json.dumps(contents))
            '''
            ct+=1
        else:
            break
response.close()
    

In [ ]:
print(query_url, response) # 200 <OK>
count = 0
for line in response.iter_lines():  # Iterate over streaming tweets
    try:
        if count > BLOCKSIZE:
            break
        post     = json.loads(line.decode('utf-8'))
        contents = [post['text'], post['user']['screen_name'], post['user']['location']]
        count   += 1
        if not isinstance(contents,int):
            print(str(contents))
        yield str(contents)
    except:
        print(line)

In [ ]:
 if not isinstance(data,int):
    if (('text' in data) and 
        ('retweeted_status' not in data) and 
        (len(data['entities']['urls'])==0) and
        (('media' not in data['entities']) or all(['url' not in m for m in data['entities']['media']]))
        ):

        print "User:",data['user']['screen_name']
        print data['text'].encode('utf-8')
        print "Hashtags:",[ht['text'] for ht in data['entities']['hashtags']]
        print "URLs:",data['entities']['urls']
        print
        if re.search("http",data['text']):
            print data

In [4]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate